In [9]:
import csv
import pandas as pd
import os

In [14]:
# Specify the path to the parent folder ("SCAI_comp")
parent_folder = '/Users/andersoncompalas/Documents/SCAI_comp'

# Specify the name of the dataset folder
# dataset_folder = 'datasets'

# Specify the name of the CSV file
file_name = 'datasets/nba_games.csv'

# Construct the full path to the CSV file
csv_file_path = os.path.join(parent_folder, dataset_folder, file_name)

In [20]:
# Initialize an empty dictionary to store the data
mydict = {}
    
# with open(file_name, mode='r') as infile:
#     reader = csv.reader(infile)
#     mydict = {rows[0]:rows[1] for rows in reader}

with open(file_name, 'r') as f:
     
    dict_reader = csv.DictReader(f)
     
    list_of_dict = list(dict_reader)
   
    print(list_of_dict)
        
# Display the first 5 rows from the dictionary
for key, value in list(mydict.items())[:5]:
    print(f'Key: {key}, Value: {value}')
    
print(mydict)


NameError: name 'DictReader' is not defined